Tables created from shoes dataframe for use in the web app. Separated this out from the main analysis for clarity.

In [43]:
import pandas as pd
import numpy as np
import pickle

In [44]:
with open('data/final_sales_history.pkl', 'rb') as picklefile:
    sales_history = pickle.load(picklefile)

In [45]:
sales_history.head()

,name,weekday,shoe_size,sale_price,sale_date_time,shoe_name,year,month,day,hour,sale_date,line,brand,size_number,colorway,image_url,original_retail,release_date,style_code,main_color
90233,Adidas AW Run Alexander Wang Black,Wednesday,0,325,2017-03-15 16:05:00,Adidas AW Run Alexander Wang Black,2017,3,15,16,2017-03-15,adidas,adidas,9.0,black/white,https://stockx.imgix.net/Adidas-AW-Run-Alexand...,230.0,2017-03-04,cm7825,basic
90243,Adidas AW Run Alexander Wang Black,Monday,0,350,2017-03-13 11:41:00,Adidas AW Run Alexander Wang Black,2017,3,13,11,2017-03-13,adidas,adidas,12.0,black/white,https://stockx.imgix.net/Adidas-AW-Run-Alexand...,230.0,2017-03-04,cm7825,basic
90244,Adidas AW Run Alexander Wang Black,Monday,0,315,2017-03-13 11:12:00,Adidas AW Run Alexander Wang Black,2017,3,13,11,2017-03-13,adidas,adidas,12.5,black/white,https://stockx.imgix.net/Adidas-AW-Run-Alexand...,230.0,2017-03-04,cm7825,basic
90245,Adidas AW Run Alexander Wang Black,Monday,0,325,2017-03-13 10:41:00,Adidas AW Run Alexander Wang Black,2017,3,13,10,2017-03-13,adidas,adidas,9.5,black/white,https://stockx.imgix.net/Adidas-AW-Run-Alexand...,230.0,2017-03-04,cm7825,basic
90246,Adidas AW Run Alexander Wang Black,Monday,0,275,2017-03-13 09:14:00,Adidas AW Run Alexander Wang Black,2017,3,13,9,2017-03-13,adidas,adidas,11.0,black/white,https://stockx.imgix.net/Adidas-AW-Run-Alexand...,230.0,2017-03-04,cm7825,basic


In [46]:
## Add a num_sales (frequency) column
sales_history['num_sales'] = sales_history.groupby('shoe_name')['shoe_name'].transform('count')

In [47]:
## Limit everything below this to shoes with over 50 sales recorded
print(len(sales_history))
sales_history = sales_history[sales_history.num_sales > 50]
print(len(sales_history))

175844
171396


In [48]:
# Creates the shoe_info csv for the flask app
info_cols = ['release_date', 'image_url', 'style_code', 'colorway', 'original_retail',
                          'main_color', 'line', 'brand']

sales_history = sales_history.set_index(['name', 'sale_date_time'])
shoe_info = sales_history[info_cols].reset_index()

# number of transactions in last month
shoe_info = shoe_info[shoe_info.sale_date_time > '2017-02-21']
shoe_info['transactions_last_month'] = shoe_info.groupby('name')['name'].transform('count')

# shoe image URL - remove extra stuff at the end
shoe_info.image_url = shoe_info['image_url'].apply(lambda x: x.split('?')[0])
shoe_info = shoe_info.drop('sale_date_time', 1)
shoe_info = shoe_info.drop_duplicates()
print(shoe_info.head())

# send to CSV
shoe_info.to_csv('data/shoe_info.csv', index = False)

                                                name release_date  \
0                 Adidas AW Run Alexander Wang Black   2017-03-04   
61          Adidas AW Run Clean Alexander Wang White   2017-03-04   
256   Adidas EQT Guidance '93 Pusha T "Black Market"   2015-11-27   
627    Adidas EQT Support 93/17 Core Black Turbo Red   2017-01-26   
1051             Adidas EQT Support ADV Triple Black   2016-07-01   

                                              image_url style_code  \
0     https://stockx.imgix.net/Adidas-AW-Run-Alexand...     cm7825   
61    https://stockx.imgix.net/Adidas-AW-Run-Clean-A...     cm7828   
256   https://stockx.imgix.net/Adidas-EQT-Guidance-9...     aq7433   
627   https://stockx.imgix.net/Adidas-EQT-Support-93...     bb1234   
1051  https://stockx.imgix.net/Adidas-EQT-Support-AD...     ba8324   

                                        colorway  original_retail main_color  \
0                                    black/white            230.0      basic   
61  

In [49]:
# For the historical sales chart 
chart_data = sales_history.reset_index()
chart_data = chart_data.set_index('sale_date_time')
chart_data['date'] = chart_data.index.date
chart_data = chart_data.reset_index()
chart_data = chart_data[['name', 'sale_date', 'sale_price']].groupby(['name', 'sale_date'])

chart_data = chart_data.aggregate(['count', 'mean', 'min', 'max']).reset_index()
chart_data.columns = chart_data.columns.droplevel(0)
chart_data = pd.DataFrame(chart_data)

# rename columns for clarity
chart_data.columns = ['name', 'date', 'volume', 'sale_mean', 'sale_min', 'sale_max']
chart_data.sale_mean = chart_data['sale_mean'].apply(lambda x: round(x))

# write to CSV
chart_data.to_csv('data/chart_data.csv', index = False)
print(chart_data.head())

                                 name        date  volume  sale_mean  \
0  Adidas AW Run Alexander Wang Black  2017-03-03       2      586.0   
1  Adidas AW Run Alexander Wang Black  2017-03-05       1      517.0   
2  Adidas AW Run Alexander Wang Black  2017-03-06       3      423.0   
3  Adidas AW Run Alexander Wang Black  2017-03-08       9      371.0   
4  Adidas AW Run Alexander Wang Black  2017-03-09      12      344.0   

   sale_min  sale_max  
0       497       675  
1       517       517  
2       300       497  
3       350       486  
4       286       400  
